In [1]:
## Load R libraries.

inLibraries = list('repr',
                   'rgdal',
                   'gstat',
                   'sp',
                   'spdep',
                   'rgeos',
                   'maptools',
                   'RColorBrewer',
                   'classInt',
                   'raster',
                   'geojsonio', 
                   'tidyverse',
                   'sjstats',
                   'naniar')
for (rpack in inLibraries) {
  if (is.element(rpack,installed.packages()[,1])){           
      #Load the library into R
      suppressMessages(library(rpack,character.only = TRUE))
    }
    else {
        print(paste("Warning:  ",rpack," is not an installed package"))
    }
}

options(repr.plot.width=15, repr.plot.height=15)


[1] "Warning:   geojsonio  is not an installed package"
[1] "Warning:   sjstats  is not an installed package"
[1] "Warning:   naniar  is not an installed package"


In [2]:
meteorites <- read.csv("data/results/meteorites.csv",stringsAsFactors=FALSE)
meteorites <- subset(meteorites, select = -c(X) )

population <- read.csv("data/results/meteorites_population.csv",stringsAsFactors=FALSE)
population <- subset(population, select = -c(X) )

landcover <- read.csv("data/results/meteorites_landcover.csv",stringsAsFactors=FALSE)
landcover <- subset(landcover, select = -c(X) )

head(meteorites)
head(population)
head(landcover)

name,id,nametype,recclass,mass_in_grams,fell_or_found,year,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4
Aachen,1,Valid,L5,21,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L
Aarhus,2,Valid,H6,720,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H
Abajo,4,Valid,H5,331,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H
Abbott,5,Valid,H3-6,21100,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H
Abee,6,Valid,EH4,107000,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL
Abernathy,7,Valid,L6,2914,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L


x,y,name,id,year,closest_city,pop,pop_year
6.08333,50.77500,Aachen,1,1880,Aachen,48688,1851
10.23333,56.18333,Aarhus,2,1951,Aarhus,51000,1900
-105.41667,26.80000,Abajo,4,1982,NA,NA,NA
-104.28333,36.30000,Abbott,5,1951,NA,NA,NA
-113.00000,54.21667,Abee,6,1952,NA,NA,NA
-101.80000,33.85000,Abernathy,7,1941,NA,NA,NA


id,name,lc_sample
1,Aachen,11
2,Aarhus,1
4,Abajo,2
5,Abbott,13
6,Abee,1
7,Abernathy,1


In [3]:
meteorites_lc <- merge(meteorites,landcover,by="id")

head(meteorites_lc)




id,name.x,nametype,recclass,mass_in_grams,fell_or_found,year,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,name.y,lc_sample
1,Aachen,Valid,L5,21,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,Aachen,11
2,Aarhus,Valid,H6,720,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,Aarhus,1
4,Abajo,Valid,H5,331,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,Abajo,2
5,Abbott,Valid,H3-6,21100,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,Abbott,13
6,Abee,Valid,EH4,107000,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,Abee,1
7,Abernathy,Valid,L6,2914,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,Abernathy,1


In [6]:
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 0] <- "Oceans/Water"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 1] <- "Cultivated Land"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 2] <- "Pasture/Land used for
Grazing"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 5] <- "Ice"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 6] <- "Tundra"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 7] <- "Wooded Tundra"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 8] <- "Boreal Forest"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 9] <- "Cool Conifer Forest"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 10] <- "Temperate Mixed Forest"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 11] <- "Temperate Decidious Forest"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 12] <- "Warm Mixed Forest"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 13] <- "Grassland/Steppe"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 14] <- "Hot Desert"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 15] <- "Scrubland"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 16] <- "Savanna"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 17] <- "Tropical Woodland"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 18] <- "Tropical Forest"
meteorites_lc["lc_sample"][meteorites_lc["lc_sample"] == 19] <- "No Data over Land"

head(meteorites_lc)

id,name.x,nametype,recclass,mass_in_grams,fell_or_found,year,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,name.y,lc_sample
1,Aachen,Valid,L5,21,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,Aachen,Temperate Decidious Forest
2,Aarhus,Valid,H6,720,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,Aarhus,Cultivated Land
4,Abajo,Valid,H5,331,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,Abajo,Pasture/Land used for Grazing
5,Abbott,Valid,H3-6,21100,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,Abbott,Grassland/Steppe
6,Abee,Valid,EH4,107000,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,Abee,Cultivated Land
7,Abernathy,Valid,L6,2914,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,Abernathy,Cultivated Land


In [7]:
write.csv(meteorites_lc,'data/results/meteorites_lc.csv')


In [4]:
megatable <- merge(meteorites_lc,population,by="id")

keep <- c('id',
          'nametype',
          'recclass',
          'mass_in_grams',
          'fell_or_found',
          'year.x',
          'latitude',
          'longitude',
          'GeoLocation',
          'group_name',
          'Chondrite.Achondrite',
          'type',
          'L3',
          'L4',
          'lc_sample',
          'closest_city',
          'pop', 
          'pop_year')

megatable2 <- subset(megatable, select = c(keep))

megatable2

id,nametype,recclass,mass_in_grams,fell_or_found,year.x,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,closest_city,pop,pop_year
1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,11,Aachen,48688,1851
2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,1,Aarhus,51000,1900
4,Valid,H5,331.0,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,2,NA,NA,NA
5,Valid,H3-6,21100.0,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,13,NA,NA,NA
6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,1,NA,NA,NA
7,Valid,L6,2914.0,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,1,NA,NA,NA
8,Valid,H,1.2,Found,1840,60.43333,22.30000,"(60.43333, 22.3)",H,Chondrite,Stony,Ordinary,H,1,NA,NA,NA
9,Valid,H4,4500.0,Found,1997,27.23944,29.83583,"(27.23944, 29.83583)",H,Chondrite,Stony,Ordinary,H,11,NA,NA,NA
10,Valid,Acapulcoite,1914.0,Fell,1976,16.88333,-99.90000,"(16.88333, -99.9)",Acapulcoite,Achondrite,Stony,Primitive,-,0,NA,NA,NA
11,Valid,L6,6700.0,Found,1989,27.50000,3.61667,"(27.5, 3.61667)",L,Chondrite,Stony,Ordinary,L,14,NA,NA,NA


In [5]:
# Creating a score to weight meteorite finds.

# I want there to be a weighting where meteorites found in rare places are "worth more" than those found 
# in obvious places.

# First looking at the population:

urban <- subset(megatable2, (!is.na(megatable$pop)))
summary(urban)
urban

       id          nametype           recclass         mass_in_grams     
 Min.   :    1   Length:239         Length:239         Min.   :       0  
 1st Qu.:10851   Class :character   Class :character   1st Qu.:     367  
 Median :16627   Mode  :character   Mode  :character   Median :    1500  
 Mean   :17644                                         Mean   :  108967  
 3rd Qu.:23758                                         3rd Qu.:    8750  
 Max.   :55584                                         Max.   :15500000  
 fell_or_found          year.x        latitude        longitude       
 Length:239         Min.   :1623   Min.   :-34.55   Min.   :-157.783  
 Class :character   1st Qu.:1893   1st Qu.: 28.50   1st Qu.:  -6.533  
 Mode  :character   Median :1934   Median : 38.18   Median :  13.167  
                    Mean   :1925   Mean   : 35.21   Mean   :  17.778  
                    3rd Qu.:1976   3rd Qu.: 46.30   3rd Qu.:  76.282  
                    Max.   :2010   Max.   : 58.55   Max.

,id,nametype,recclass,mass_in_grams,fell_or_found,year.x,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,closest_city,pop,pop_year
1,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,11,Aachen,48688,1851
2,2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,1,Aarhus,51000,1900
415,425,Valid,Unknown,367.0,Fell,1769,50.66667,2.33333,"(50.66667, 2.33333)",Unknown,-,-,-,-,1,Saint-Omer,20000,1300
436,446,Valid,CR2-an,160.0,Fell,1957,24.41667,39.51667,"(24.41667, 39.51667)",CR,Chondrite,Stony,Carbonaceous,CR,2,Medina,27500,1900
451,462,Valid,L6,3200.0,Fell,1873,36.23333,37.13333,"(36.23333, 37.13333)",L,Chondrite,Stony,Ordinary,L,2,Aleppo,99383,1856
452,463,Valid,H5,908.0,Fell,1860,44.88333,8.75000,"(44.88333, 8.75)",H,Chondrite,Stony,Ordinary,H,1,Alessandria,22994,1848
455,466,Valid,L6,228000.0,Fell,1883,45.26667,10.15000,"(45.26667, 10.15)",L,Chondrite,Stony,Ordinary,L,1,Cremona,28328,1851
485,2301,Valid,L6,367.0,Fell,1822,47.46667,-0.55000,"(47.46667, -0.55)",L,Chondrite,Stony,Ordinary,L,1,Angers,25442,1809
497,2315,Valid,L6,6.1,Found,1938,39.75000,-105.20000,"(39.75, -105.2)",L,Chondrite,Stony,Ordinary,L,1,Denver,272000,1925
499,2318,Valid,LL6,15000.0,Fell,1914,53.58333,-2.71667,"(53.58333, -2.71667)",LL,Chondrite,Stony,Ordinary,LL,1,Preston,120000,1900


In [6]:
# 239 of the hits were near an urban area.  Honestly, I don't believe 
# that's going to affect my results very much.  

In [7]:
# Next, landcover. let's create and normalize the weights column.  
# This will assign a weight to each LC type, based on it's percentage in the 1990 land cover raster, 
# which is then normalized.

lc1990name <- "data/lc/historic_landcover_hd_1990.asc"
lc1990 <- raster(lc1990name,values=TRUE)

landcover_counts<- freq(lc1990)

total <- 259200

for (i in 1:18){
    lc_weight <- (landcover_counts[i,2]/total)*100
    megatable2$weights[megatable2$lc_sample == landcover_counts[i,1]] <- lc_weight

}

normalize <- function(x) {
return ((x - min(x)) / (max(x) - min(x)))
}

unique(megatable2$weights)

megatable2$weights<-normalize(megatable2$weights)

unique(megatable2$weights)

[1]  0.3364198  2.2827932  4.5821759  1.4336420 66.3206019  2.2241512
 [7]  0.3406636  4.0671296  1.0501543  1.0856481  0.5412809  0.3572531
[13]  0.6041667  0.7133488  1.8429784  0.7391975

[1] 0.000000e+00 2.949758e-02 6.434506e-02 1.662856e-02 1.000000e+00
 [6] 2.860885e-02 6.431583e-05 5.653946e-02 1.081675e-02 1.135467e-02
[11] 3.104700e-03 3.157322e-04 4.057744e-03 5.712415e-03 2.283212e-02
[16] 6.104157e-03

In [8]:
colnames(megatable2)
head(megatable2)

[1] "id"                   "nametype"             "recclass"            
 [4] "mass_in_grams"        "fell_or_found"        "year.x"              
 [7] "latitude"             "longitude"            "GeoLocation"         
[10] "group_name"           "Chondrite.Achondrite" "type"                
[13] "L3"                   "L4"                   "lc_sample"           
[16] "closest_city"         "pop"                  "pop_year"            
[19] "weights"

id,nametype,recclass,mass_in_grams,fell_or_found,year.x,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,closest_city,pop,pop_year,weights
1,Valid,L5,21,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,11,Aachen,48688,1851,0.00000000
2,Valid,H6,720,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,1,Aarhus,51000,1900,0.02949758
4,Valid,H5,331,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,2,NA,NA,NA,0.06434506
5,Valid,H3-6,21100,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,13,NA,NA,NA,0.01662856
6,Valid,EH4,107000,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,1,NA,NA,NA,0.02949758
7,Valid,L6,2914,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,1,NA,NA,NA,0.02949758


In [11]:
# Want to convert some columns to factors, but first need to clean up some things that are not NA...

megatable2$Chondrite.Achondrite[megatable2$Chondrite.Achondrite == "-"] <-NA
megatable2$type[megatable2$type == "-"] <- NA
megatable2$L3[megatable2$L3 == "-"] <- NA
megatable2$L4[megatable2$L4 == "-"] <- NA
megatable2$L4[megatable2$L4 == ""] <- NA


head(megatable2,6L)
str(megatable2)

id,nametype,recclass,mass_in_grams,fell_or_found,year.x,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,closest_city,pop,pop_year,weights
1,Valid,L5,21,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,11,Aachen,48688,1851,0.00000000
2,Valid,H6,720,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,1,Aarhus,51000,1900,0.02949758
4,Valid,H5,331,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,2,NA,NA,NA,0.06434506
5,Valid,H3-6,21100,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,13,NA,NA,NA,0.01662856
6,Valid,EH4,107000,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,1,NA,NA,NA,0.02949758
7,Valid,L6,2914,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,1,NA,NA,NA,0.02949758


'data.frame':	9897 obs. of  19 variables:
 $ id                  : int  1 2 4 5 6 7 8 9 10 11 ...
 $ nametype            : chr  "Valid" "Valid" "Valid" "Valid" ...
 $ recclass            : chr  "L5" "H6" "H5" "H3-6" ...
 $ mass_in_grams       : num  21 720 331 21100 107000 ...
 $ fell_or_found       : chr  "Fell" "Fell" "Found" "Found" ...
 $ year.x              : int  1880 1951 1982 1951 1952 1941 1840 1997 1976 1989 ...
 $ latitude            : num  50.8 56.2 26.8 36.3 54.2 ...
 $ longitude           : num  6.08 10.23 -105.42 -104.28 -113 ...
 $ GeoLocation         : chr  "(50.775, 6.08333)" "(56.18333, 10.23333)" "(26.8, -105.41667)" "(36.3, -104.28333)" ...
 $ group_name          : chr  "L" "H" "H" "H" ...
 $ Chondrite.Achondrite: chr  "Chondrite" "Chondrite" "Chondrite" "Chondrite" ...
 $ type                : chr  "Stony" "Stony" "Stony" "Stony" ...
 $ L3                  : chr  "Ordinary" "Ordinary" "Ordinary" "Ordinary" ...
 $ L4                  : chr  "L" "H" "H" "H" ...
 $ l

In [12]:
#converting some columns to factors:

megatable2$recclass <- factor(megatable2$recclass)
megatable2$fell_or_found <- factor(megatable2$fell_or_found)
megatable2$group_name <- factor(megatable2$group_name)
megatable2$Chondrite.Achondrite <- factor(megatable2$Chondrite.Achondrite)
megatable2$type <- factor(megatable2$type)
megatable2$L3 <- factor(megatable2$L3)
megatable2$L4 <- factor(megatable2$L4)
megatable2$lc_sample <- factor(megatable2$lc_sample)

str(megatable2)

'data.frame':	9897 obs. of  19 variables:
 $ id                  : int  1 2 4 5 6 7 8 9 10 11 ...
 $ nametype            : chr  "Valid" "Valid" "Valid" "Valid" ...
 $ recclass            : Factor w/ 311 levels "Acapulcoite",..: 220 128 122 90 52 224 70 116 1 224 ...
 $ mass_in_grams       : num  21 720 331 21100 107000 ...
 $ fell_or_found       : Factor w/ 2 levels "Fell","Found": 1 1 2 2 1 2 2 2 1 2 ...
 $ year.x              : int  1880 1951 1982 1951 1952 1941 1840 1997 1976 1989 ...
 $ latitude            : num  50.8 56.2 26.8 36.3 54.2 ...
 $ longitude           : num  6.08 10.23 -105.42 -104.28 -113 ...
 $ GeoLocation         : chr  "(50.775, 6.08333)" "(56.18333, 10.23333)" "(26.8, -105.41667)" "(36.3, -104.28333)" ...
 $ group_name          : Factor w/ 53 levels "Acapulcoite",..: 40 22 22 22 17 40 22 22 1 40 ...
 $ Chondrite.Achondrite: Factor w/ 2 levels "Achondrite","Chondrite": 2 2 2 2 2 2 2 2 1 2 ...
 $ type                : Factor w/ 3 levels "Iron","Stony",..: 2 2 2 2 2 

In [13]:
head(megatable2)

id,nametype,recclass,mass_in_grams,fell_or_found,year.x,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,closest_city,pop,pop_year,weights
1,Valid,L5,21,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,11,Aachen,48688,1851,0.00000000
2,Valid,H6,720,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,1,Aarhus,51000,1900,0.02949758
4,Valid,H5,331,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,2,NA,NA,NA,0.06434506
5,Valid,H3-6,21100,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,13,NA,NA,NA,0.01662856
6,Valid,EH4,107000,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,1,NA,NA,NA,0.02949758
7,Valid,L6,2914,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,1,NA,NA,NA,0.02949758


In [14]:
megatable2 <- subset(megatable2, select = -c(nametype,GeoLocation,closest_city,pop,pop_year) )

head(megatable2)

id,recclass,mass_in_grams,fell_or_found,year.x,latitude,longitude,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,weights
1,L5,21,Fell,1880,50.77500,6.08333,L,Chondrite,Stony,Ordinary,L,11,0.00000000
2,H6,720,Fell,1951,56.18333,10.23333,H,Chondrite,Stony,Ordinary,H,1,0.02949758
4,H5,331,Found,1982,26.80000,-105.41667,H,Chondrite,Stony,Ordinary,H,2,0.06434506
5,H3-6,21100,Found,1951,36.30000,-104.28333,H,Chondrite,Stony,Ordinary,H,13,0.01662856
6,EH4,107000,Fell,1952,54.21667,-113.00000,EH,Chondrite,Stony,Enstatite,EH-EL,1,0.02949758
7,L6,2914,Found,1941,33.85000,-101.80000,L,Chondrite,Stony,Ordinary,L,1,0.02949758


In [15]:
colnames(megatable2) <- c('id',
                            'recclass',
                            'mass', 
                            'fell_found',
                            'year',
                            'latitude',
                            'longitude',
                            'group',
                            'chondrite',
                            'type',
                            'lvl3',
                            'lvl4',
                            'lc_sample',
                            'weights')

head(megatable2)

id,recclass,mass,fell_found,year,latitude,longitude,group,chondrite,type,lvl3,lvl4,lc_sample,weights
1,L5,21,Fell,1880,50.77500,6.08333,L,Chondrite,Stony,Ordinary,L,11,0.00000000
2,H6,720,Fell,1951,56.18333,10.23333,H,Chondrite,Stony,Ordinary,H,1,0.02949758
4,H5,331,Found,1982,26.80000,-105.41667,H,Chondrite,Stony,Ordinary,H,2,0.06434506
5,H3-6,21100,Found,1951,36.30000,-104.28333,H,Chondrite,Stony,Ordinary,H,13,0.01662856
6,EH4,107000,Fell,1952,54.21667,-113.00000,EH,Chondrite,Stony,Enstatite,EH-EL,1,0.02949758
7,L6,2914,Found,1941,33.85000,-101.80000,L,Chondrite,Stony,Ordinary,L,1,0.02949758


In [16]:
write.csv(megatable2,'data/results/meteorites_unweighted.csv')

In [17]:
#I will create a new sample, using the weights while allowing replacements, 
# so that I can get the correct ratios, even if it means some meteorites will be recorded more than once.

set.seed(1234)

megatable3 <- megatable2[sample(seq_len(nrow(megatable2)),nrow(megatable2),replace = TRUE, prob=megatable2$weights),]

head(megatable3)
dim(megatable3)

write.csv(megatable3,'data/results/meteorites_weighted.csv')

,id,recclass,mass,fell_found,year,latitude,longitude,group,chondrite,type,lvl3,lvl4,lc_sample,weights
6854,24234,H4,27700,Found,1934,40.35000,-103.95000,H,Chondrite,Stony,Ordinary,H,1,0.02949758
4812,15495,L5,2405,Fell,1955,38.18333,15.56667,L,Chondrite,Stony,Ordinary,L,0,1.00000000
4859,16712,H6,1800,Found,1948,38.50000,-101.10000,H,Chondrite,Stony,Ordinary,H,2,0.06434506
4811,15495,L5,2405,Fell,1955,38.18333,15.56667,L,Chondrite,Stony,Ordinary,L,0,1.00000000
3580,11208,L6,8000,Fell,1881,46.35000,30.58333,L,Chondrite,Stony,Ordinary,L,0,1.00000000
6338,23625,L4,1070,Found,1941,33.43333,-101.75000,L,Chondrite,Stony,Ordinary,L,1,0.02949758


[1] 9897   14